In [19]:
import os
import sys
import json
import numpy as np
from scipy import stats
from utilities import Hyperparameters
from datetime import datetime
import tensorflow as tf
import matplotlib.pyplot as plt

In [33]:
SCRIPT_DIR_PATH = r"X:\Project\ETH\EthereumPriceNet-master"
DATA_DIR_REL_PATH = 'data/'
RAW_DATA_DIR_REL_PATH = 'data/raw/'
DATA_DIR_ABS_PATH = os.path.join(SCRIPT_DIR_PATH, DATA_DIR_REL_PATH)
RAW_DATA_DIR_ABS_PATH = os.path.join(SCRIPT_DIR_PATH, RAW_DATA_DIR_REL_PATH)

In [39]:

RESULTS_DIR_REL_PATH = 'results/' #+ datetime.now().isoformat(' ', 'seconds') + '/'

RESULTS_DIR_ABS_PATH = os.path.join(SCRIPT_DIR_PATH, RESULTS_DIR_REL_PATH)

In [22]:
def load_raw_data():
    f_daily_avg_gas_limit = open(os.path.join(RAW_DATA_DIR_ABS_PATH, 'dailyavggaslimit.json'), 'r')
    f_daily_avg_gas_price = open(os.path.join(RAW_DATA_DIR_ABS_PATH, 'dailyavggasprice.json'), 'r')
    f_daily_gas_used = open(os.path.join(RAW_DATA_DIR_ABS_PATH, 'dailygasused.json'), 'r')
    f_daily_txn_fee = open(os.path.join(RAW_DATA_DIR_ABS_PATH, 'dailytxnfee.json'), 'r')
    f_eth_daily_market_cap = open(os.path.join(RAW_DATA_DIR_ABS_PATH, 'ethdailymarketcap.json'), 'r')
    f_eth_daily_price = open(os.path.join(RAW_DATA_DIR_ABS_PATH, 'ethdailyprice.json'), 'r')

    json_daily_avg_gas_limit = json.load(f_daily_avg_gas_limit)
    json_daily_avg_gas_price = json.load(f_daily_avg_gas_price)
    json_daily_gas_used = json.load(f_daily_gas_used)
    json_daily_txn_fee = json.load(f_daily_txn_fee)
    json_eth_daily_market_cap = json.load(f_eth_daily_market_cap)
    json_eth_daily_price = json.load(f_eth_daily_price)

    f_daily_avg_gas_limit.close()
    f_daily_avg_gas_price.close()
    f_daily_gas_used.close()
    f_daily_txn_fee.close()
    f_eth_daily_market_cap.close()
    f_eth_daily_price.close()

    daily_avg_gas_limit_data = np.asarray([[d['unixTimeStamp'], d['gasLimit']] for d in json_daily_avg_gas_limit['result']][8:], dtype=np.float64)
    daily_avg_gas_price_data = np.asarray([[d['unixTimeStamp'], d['avgGasPrice_Wei']] for d in json_daily_avg_gas_price['result']][8:], dtype=np.float64)
    daily_gas_used_data = np.asarray([[d['unixTimeStamp'], d['gasUsed']] for d in json_daily_gas_used['result']][8:], dtype=np.float64)
    daily_txn_fee_data = np.asarray([[d['unixTimeStamp'], d['transactionFee_Eth']] for d in json_daily_txn_fee['result']][8:], dtype=np.float64)
    eth_daily_market_cap_data = np.asarray([[d['unixTimeStamp'], d['marketCap']] for d in json_eth_daily_market_cap['result']][8:], dtype=np.float64)
    eth_daily_price_data = np.asarray([[d['unixTimeStamp'], d['value']] for d in json_eth_daily_price['result']][8:], dtype=np.float64)

    print(daily_avg_gas_limit_data.shape)

    return (daily_avg_gas_limit_data[:, 1], daily_avg_gas_price_data[:, 1], daily_gas_used_data[:, 1], daily_txn_fee_data[:, 1], eth_daily_market_cap_data[:, 1], eth_daily_price_data[:, 1])

In [23]:
def get_price_ffts(hps, eth_daily_price_data):
    windows = []
    for i in range(0, eth_daily_price_data.shape[0] - hps.fft_window_size + 1):
        window = eth_daily_price_data[i:i + hps.fft_window_size]
        windows += [stats.zscore(window)]
    windows = np.vstack(windows)
    return np.abs(np.fft.fft(windows))

In [24]:
def get_preprocessed_data(hps, full_sequence):
    windows, y = [], []
    for i in range(0, full_sequence.shape[0] - hps.sequence_length + 1 - hps.prediction_window_size):
        window = full_sequence[i:i + hps.sequence_length, :]
        windows += [window]
        prediction_window = full_sequence[i + hps.sequence_length:i + hps.sequence_length + hps.prediction_window_size, 5]
        y += [[
            100*(np.amin(prediction_window)/window[-1, 5]-1),
            100*(np.mean(prediction_window)/window[-1, 5]-1),
            100*(np.amax(prediction_window)/window[-1, 5]-1)
        ]]
    return (np.stack(windows), np.array(y))

In [25]:
hps = Hyperparameters()

raw_data = load_raw_data()
eth_daily_price_data = raw_data[-1]
price_ffts = get_price_ffts(hps, eth_daily_price_data)
full_sequence = np.concatenate((np.stack(raw_data, axis=1)[hps.fft_window_size - 1:, :], price_ffts), axis=1)
X, y = get_preprocessed_data(hps, full_sequence)

print('X.shape = ', X.shape)
print('y.shape = ', y.shape)

np.save(os.path.join(DATA_DIR_ABS_PATH, 'X.npy'), X)
np.save(os.path.join(DATA_DIR_ABS_PATH, 'y.npy'), y)

(2656, 2)
X.shape =  (2623, 14, 20)
y.shape =  (2623, 3)


In [26]:
def load_data():
    X = np.load(os.path.join(DATA_DIR_ABS_PATH, 'X.npy'))
    y = np.load(os.path.join(DATA_DIR_ABS_PATH, 'y.npy'))
    return (X, y)

In [27]:
def generate_model(hps):
    inputs = tf.keras.Input(shape=(hps.sequence_length, 6 + hps.fft_window_size), name='lstm_inputs')
    batch_norm = tf.keras.layers.BatchNormalization()(inputs)
    lstm1 = tf.keras.layers.LSTM(units=hps.lstm1_units, return_sequences=True, kernel_regularizer=hps.lstm1_regularizer)(batch_norm)
    lstm2 = tf.keras.layers.LSTM(units=hps.lstm2_units, kernel_regularizer=hps.lstm2_regularizer)(lstm1)
    dense = tf.keras.layers.Dense(units=hps.dense_units, activation='tanh', kernel_regularizer=hps.dense_regularizer)(lstm2)
    outputs = tf.keras.layers.Dense(units=3)(dense)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=hps.learning_rate), metrics=['mae'])
    return model

In [28]:
def draw_results(y, predictions, title):
    plt.figure()
    plt.plot(y[:, 0], label='actual')
    plt.plot(predictions[:, 0], label='predicted')
    plt.xlabel('day')
    plt.ylabel('%% change in price')
    plt.title(title + ' (min)')
    plt.legend()
    plt.savefig(os.path.join(RESULTS_DIR_ABS_PATH, title + '_min.png'), dpi=600, format='png')

    plt.figure()
    plt.plot(y[:, 1], label='actual')
    plt.plot(predictions[:, 1], label='predicted')
    plt.xlabel('day')
    plt.ylabel('%% change in price')
    plt.title(title + ' (avg)')
    plt.legend()
    plt.savefig(os.path.join(RESULTS_DIR_ABS_PATH, title + '_avg.png'), dpi=600, format='png')

    plt.figure()
    plt.plot(y[:, 2], label='actual')
    plt.plot(predictions[:, 2], label='predicted')
    plt.xlabel('day')
    plt.ylabel('%% change in price')
    plt.title(title + ' (max)')
    plt.legend()
    plt.savefig(os.path.join(RESULTS_DIR_ABS_PATH, title + '_max.png'), dpi=600, format='png')










    

In [29]:
def draw_history(history):
    plt.figure()
    plt.plot(history.history['loss'], label='loss')
    plt.xlabel('epoch')
    plt.ylabel('loss (mse)')
    plt.title('loss')
    plt.legend()
    plt.savefig(os.path.join(RESULTS_DIR_ABS_PATH, 'loss.png'), dpi=600, format='png')

    plt.figure()
    plt.plot(history.history['val_loss'], label='val loss')
    plt.xlabel('epoch')
    plt.ylabel('loss (mse)')
    plt.title('val loss')
    plt.legend()
    plt.savefig(os.path.join(RESULTS_DIR_ABS_PATH, 'val_loss.png'), dpi=600, format='png')

In [30]:
def split_data(hps, X, y):
    index = int(hps.train_split * X.shape[0])
    return (X[:index], y[:index], X[index:], y[index:])

In [ ]:

    
with open(os.path.join(RESULTS_DIR_ABS_PATH, 'output.txt'), 'w') as f:
    X, y = load_data()
    X_train, y_train, X_val, y_val = split_data(hps, X, y)

    model = generate_model(hps)
    tf.keras.utils.plot_model(model, os.path.join(RESULTS_DIR_ABS_PATH, 'model.png'), show_shapes=True)
    model.summary()

    #if (len(sys.argv) > 1):
        #model.load_weights(sys.argv[1])

    history = model.fit(
        X_train,
        y_train,
        epochs=hps.epochs,
        batch_size=hps.batch_size,
        validation_data=(X_val, y_val)
    )

    model.save_weights(os.path.join(RESULTS_DIR_ABS_PATH, 'weights.h5'))

    train_predictions = model.predict(X_train)
    val_predictions = model.predict(X_val)

    draw_results(y_train, train_predictions, 'train')
    draw_results(y_val, val_predictions, 'validation')
    draw_history(history)

    f.write('loss: {}\nmae: {}\nval loss: {}\nval mae: {}\n'.format(
        history.history['loss'][-1],
        history.history['mae'][-1],
        history.history['val_loss'][-1],
        history.history['val_mae'][-1]
    ))
    f.write('val rho (min): {}\nval rho (avg): {}\nval rho (max): {}'.format(
        np.corrcoef(y_val[:, 0], val_predictions[:, 0])[0, 1],
        np.corrcoef(y_val[:, 1], val_predictions[:, 1])[0, 1],
        np.corrcoef(y_val[:, 2], val_predictions[:, 2])[0, 1]
    ))

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_inputs (InputLayer)    [(None, 14, 20)]          0         
                                                                 
 batch_normalization_4 (Batc  (None, 14, 20)           80        
 hNormalization)                                                 
                                                                 
 lstm_8 (LSTM)               (None, 14, 256)           283648    
                                                                 
 lstm_9 (LSTM)               (None, 256)               525312    
                                                                 
 dense_8 (Dense)             (None, 512)               131584    
                                                                 
 dense_9 (Dense)             (None, 3)                 1539      
                                                           

10/10 [==============================] - 7s 695ms/step - loss: 115.8261 - mae: 6.9791 - val_loss: 110.9443 - val_mae: 7.7297
Epoch 52/1000
10/10 [==============================] - 8s 766ms/step - loss: 112.5884 - mae: 6.8946 - val_loss: 108.0504 - val_mae: 7.5617
Epoch 53/1000
10/10 [==============================] - 7s 678ms/step - loss: 112.7486 - mae: 6.8963 - val_loss: 110.4251 - val_mae: 7.6945
Epoch 54/1000
10/10 [==============================] - 7s 692ms/step - loss: 111.0880 - mae: 6.8342 - val_loss: 115.2952 - val_mae: 7.9592
Epoch 55/1000
10/10 [==============================] - 7s 740ms/step - loss: 111.7738 - mae: 6.8237 - val_loss: 114.6477 - val_mae: 7.9162
Epoch 56/1000
10/10 [==============================] - 6s 652ms/step - loss: 110.3382 - mae: 6.8576 - val_loss: 118.7203 - val_mae: 8.0661
Epoch 57/1000
10/10 [==============================] - 8s 775ms/step - loss: 106.4229 - mae: 6.6812 - val_loss: 120.1894 - val_mae: 8.1352
Epoch 58/1000
10/10 [====================

10/10 [==============================] - 8s 774ms/step - loss: 61.3735 - mae: 4.9297 - val_loss: 127.9478 - val_mae: 8.4637
Epoch 111/1000
10/10 [==============================] - 7s 661ms/step - loss: 63.2414 - mae: 4.9996 - val_loss: 138.3820 - val_mae: 8.9874
Epoch 112/1000
10/10 [==============================] - 9s 904ms/step - loss: 60.6426 - mae: 4.9197 - val_loss: 125.1189 - val_mae: 8.3503
Epoch 113/1000
10/10 [==============================] - 7s 675ms/step - loss: 59.6117 - mae: 4.8260 - val_loss: 144.2682 - val_mae: 9.2357
Epoch 114/1000
10/10 [==============================] - 9s 865ms/step - loss: 58.9121 - mae: 4.7965 - val_loss: 123.6247 - val_mae: 8.2752
Epoch 115/1000
10/10 [==============================] - 7s 667ms/step - loss: 58.4370 - mae: 4.8105 - val_loss: 132.4860 - val_mae: 8.6646
Epoch 116/1000
10/10 [==============================] - 9s 876ms/step - loss: 58.1884 - mae: 4.7876 - val_loss: 132.3649 - val_mae: 8.6552
Epoch 117/1000
10/10 [====================

10/10 [==============================] - 8s 770ms/step - loss: 39.2788 - mae: 3.8165 - val_loss: 111.3618 - val_mae: 7.6701
Epoch 170/1000
10/10 [==============================] - 6s 644ms/step - loss: 40.5693 - mae: 3.9168 - val_loss: 140.8997 - val_mae: 9.1217
Epoch 171/1000
10/10 [==============================] - 8s 813ms/step - loss: 39.9044 - mae: 3.8810 - val_loss: 114.2822 - val_mae: 7.8361
Epoch 172/1000
10/10 [==============================] - 7s 662ms/step - loss: 39.2542 - mae: 3.8400 - val_loss: 115.6150 - val_mae: 7.9435
Epoch 173/1000
10/10 [==============================] - 8s 843ms/step - loss: 40.1605 - mae: 3.8857 - val_loss: 123.1168 - val_mae: 8.2503
Epoch 174/1000
10/10 [==============================] - 7s 672ms/step - loss: 40.6602 - mae: 3.9046 - val_loss: 124.8435 - val_mae: 8.3460
Epoch 175/1000
10/10 [==============================] - 7s 713ms/step - loss: 37.5393 - mae: 3.7067 - val_loss: 124.5513 - val_mae: 8.3167
Epoch 176/1000
10/10 [====================

10/10 [==============================] - 7s 698ms/step - loss: 28.8333 - mae: 3.1948 - val_loss: 113.2741 - val_mae: 7.7111
Epoch 229/1000
10/10 [==============================] - 7s 718ms/step - loss: 28.5159 - mae: 3.1815 - val_loss: 120.8125 - val_mae: 8.1485
Epoch 230/1000
10/10 [==============================] - 8s 749ms/step - loss: 28.6496 - mae: 3.1957 - val_loss: 115.6615 - val_mae: 7.9161
Epoch 231/1000
10/10 [==============================] - 6s 641ms/step - loss: 27.9604 - mae: 3.1314 - val_loss: 112.7276 - val_mae: 7.7526
Epoch 232/1000
10/10 [==============================] - 8s 816ms/step - loss: 28.7305 - mae: 3.2310 - val_loss: 114.1576 - val_mae: 7.7631
Epoch 233/1000
10/10 [==============================] - 7s 674ms/step - loss: 29.0156 - mae: 3.2239 - val_loss: 114.7749 - val_mae: 7.7315
Epoch 234/1000
10/10 [==============================] - 8s 808ms/step - loss: 28.6569 - mae: 3.1800 - val_loss: 115.0922 - val_mae: 7.8652
Epoch 235/1000
10/10 [====================

10/10 [==============================] - 7s 715ms/step - loss: 23.4463 - mae: 2.8446 - val_loss: 110.3987 - val_mae: 7.5640
Epoch 288/1000
10/10 [==============================] - 6s 634ms/step - loss: 23.8858 - mae: 2.9154 - val_loss: 113.6626 - val_mae: 7.7421
Epoch 289/1000
10/10 [==============================] - 7s 745ms/step - loss: 22.5711 - mae: 2.7808 - val_loss: 110.5599 - val_mae: 7.5491
Epoch 290/1000
10/10 [==============================] - 6s 620ms/step - loss: 24.3906 - mae: 2.8997 - val_loss: 109.9272 - val_mae: 7.4848
Epoch 291/1000
10/10 [==============================] - 8s 769ms/step - loss: 24.0034 - mae: 2.9405 - val_loss: 111.5998 - val_mae: 7.7594
Epoch 292/1000
10/10 [==============================] - 7s 675ms/step - loss: 23.2745 - mae: 2.8245 - val_loss: 119.4392 - val_mae: 8.0400
Epoch 293/1000
10/10 [==============================] - 7s 739ms/step - loss: 22.9463 - mae: 2.8378 - val_loss: 109.1963 - val_mae: 7.5018
Epoch 294/1000
10/10 [====================

10/10 [==============================] - 7s 712ms/step - loss: 18.9165 - mae: 2.4979 - val_loss: 107.9825 - val_mae: 7.4633
Epoch 347/1000
10/10 [==============================] - 8s 765ms/step - loss: 18.8781 - mae: 2.4757 - val_loss: 109.8387 - val_mae: 7.5881
Epoch 348/1000
10/10 [==============================] - 7s 658ms/step - loss: 18.9322 - mae: 2.4908 - val_loss: 109.9342 - val_mae: 7.6075
Epoch 349/1000
10/10 [==============================] - 8s 807ms/step - loss: 18.4762 - mae: 2.4616 - val_loss: 107.6379 - val_mae: 7.5024
Epoch 350/1000
10/10 [==============================] - 7s 650ms/step - loss: 19.3538 - mae: 2.5402 - val_loss: 113.8948 - val_mae: 7.7402
Epoch 351/1000
10/10 [==============================] - 8s 822ms/step - loss: 18.7455 - mae: 2.5026 - val_loss: 112.9084 - val_mae: 7.6820
Epoch 352/1000
10/10 [==============================] - 7s 648ms/step - loss: 18.0761 - mae: 2.4121 - val_loss: 111.8903 - val_mae: 7.7055
Epoch 353/1000
10/10 [====================